In [1]:
import numpy as np
import functools
import pandas as pd
import os
import copy
import operator

In [2]:
#df = pd.read_csv('./data/partition/ForecastDataforTesting_201712_6.csv')
#df = df.groupby(['xid', 'yid', 'date_id', 'hour']).mean()
#df = df.reset_index()
#df = df.drop('model', axis=1)
#df_city = pd.read_csv('data/CityData.csv')
#df['hour'] = df['hour']*60-180

In [3]:
df = pd.read_csv('./data/partition/In_situMeasurementforTraining_201712_1.csv')

In [4]:
df['hour'] = df['hour']*60-180

In [5]:
df_city = pd.read_csv('./data/CityData.csv')

In [6]:
def find_neighbors(x, limit_x=[1, 548], limit_y=[1, 421]):
    xx = [(x[0]+i_i, x[1]+i_j) for i_i in [-1, 0, 1] for i_j in [-1, 0, 1] if abs(i_i)+abs(i_j)==1]
    
    xx = list(filter(lambda i: i[0]>=limit_x[0] and i[0]<=limit_x[1] and i[1]>=limit_y[0] and i[1]<=limit_y[1], xx))
    return xx

In [7]:
def pointwise_compare(x):
    x2 = np.concatenate([[np.nan], x[:-1]])
    return x!=x2

In [8]:
def _reconcile(s1, s2):
    pts = np.unique(np.sort(np.concatenate([s1._x, s2._x])))
    # Handle case when endpoints are inf
    cpts = pts.copy()
    cpts[0] = min(np.min(cpts[1:]), 0.) - 1
    cpts[-1] = max(np.max(cpts[:-1]), 0.) + 1
    mps = (cpts[1:] + cpts[:-1]) / 2.
    return [(pts, s(mps)) for s in (s1, s2)]


def _same_support(s1, s2):
    return np.all(s1._x[[0, -1]] == s2._x[[0, -1]])


def require_compatible(f):
    @functools.wraps(f)
    def wrapper(self, other, *args, **kwargs):
        if isinstance(other, StepFunction) and not _same_support(self, other):
            raise TypeError("Step functions have different support: %s vs. %s" % (
                self._x[[0, -1]], other._x[[0, -1]]))
        return f(self, other, *args, **kwargs)
    return wrapper


class StepFunction:
    '''A step function.'''

    def __init__(self, x, y):
        '''Initialize step function with breakpoints x and function values y.
        x and y are arrays such that
            f(z) = y[k], x[k] <= z < x[k + 1], 0 <= k < K.
        Thus, len(x) == len(y) + 1 and domain of f is (x[0], x[K]).
        '''
        if len(x) != 1 + len(y):
            raise RuntimeError("len(x) != 1 + len(y)")
        self._x = np.array(x)
        self._y = np.array(y, dtype='float')
        self._compress()

    @property
    def K(self):
        '''The number of steps.'''
        return len(self._y)

    def _compress(self):
        # Combine steps which have equal values
     #   ny = np.concatenate([[np.nan], self._y, [np.nan]])
     #   ys = np.diff(ny) != 0
        ys = np.concatenate([pointwise_compare(self._y), [True]])
        self._x = self._x[ys]
        self._y = self._y[ys[:-1]]

    def _binary_op(self, other, op, desc):
        if isinstance(other, StepFunction):
            (s1_x, s1_y), (s2_x, s2_y) = _reconcile(self, other)
            return StepFunction(s1_x, op(s1_y, s2_y))
        # Fall back to normal semantics otherwise
        return StepFunction(self._x, op(self._y, other))

    def __add__(self, other):
        return self._binary_op(other, operator.add, "add")

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        return self._binary_op(other, operator.sub, "subtract")

    def __rsub__(self, other):
        return -self + other

    def __mul__(self, other):
        return self._binary_op(other, operator.mul, "multiply")

    def __rmul__(self, other):
        return self * other

    def __div__(self, other):
        return self._binary_op(other, operator.div, "divide")

    def __rdiv__(self, other):
        return (self ** -1) * other

    # Unary operations

    def __neg__(self):
        return StepFunction(self._x, -self._y)

    def __pow__(self, p):
        return StepFunction(self._x, pow(self._y, p))

    def __abs__(self):
        return StepFunction(self._x, abs(self._y))

    # Equality and comparison operators

    @require_compatible
    def __eq__(self, other):
        return (np.array_equal(self._x, other._x) and 
                np.array_equal(self._y, other._y))

    @require_compatible
    def __lt__(self, other):
        diff = other - self
        return np.all(diff._y > 0)

    @require_compatible
    def __le__(self, other):
        diff = other - self
        return np.all(diff._y >= 0)

    @require_compatible
    def __gt__(self, other):
        return -self < -other

    @require_compatible
    def __ge__(self, other):
        return -self <= -other

    def __call__(self, s):
        return self._y[np.searchsorted(self._x, s, side="right") - 1]

    def __repr__(self):
        return "StepFunction(x=%s, y=%s)" % (repr(self._x), repr(self._y))

    def integral(self):
        nz = self._y != 0
        d = np.diff(self._x)
        return (d[nz] * self._y[nz]).sum()
    
    def find_interval_value(self, x_interval):
        # [a, b)
        y = np.arange(np.argwhere(self._x<=x_interval[0])[-1], np.argwhere(self._x<x_interval[1])[-1]+1)
        yy = self._y[y]
        yyy = np.sort(np.unique(np.append(self._x[y], x_interval)))
        yyy = yyy[np.all(np.array([yyy>=x_interval[0], yyy<=x_interval[1]]), axis=0)]
        return np.array([yyy, yy])
   
    def extend_domain(self, new_value):
 
        if new_value < self._x[0]:
            self._x = np.insert(self._x, 0, new_value)
            self._y = np.insert(self._y, 0, float("Inf"))
            self.__init__(self._x, self._y)
        elif new_value > self._x[0]:
            self._x = np.append(self._x, new_value)
            self._y = np.append(self._y, float("Inf"))
            #self.__init__(self._x, self._y)
        else:
            pass
            

In [9]:
def hvdistance(x, y):
    return np.sum(np.abs(np.array(x) - np.array(y)))

In [10]:
class vertex_i:
    def __init__(self, vi, vs, td, ta):
        self.vi = vi
        self.Ti = np.array([hvdistance(vi, vs)*2, ta])
        self.Si = np.array([ta, ta])
        self.TAUi = None
        self.negihbor_Sij = {}
        self.ta = ta
        self.td = td
        
    #def update_Si(self):
        
    def update_TAUi_Si(self, interval_x):
        f = StepFunction(self.gi_d, self.gi_v)
        y = f.find_interval_value(interval_x)
        self.TAUi = y[1].min()
        # self.TAUi2 = y[1].min() + hvdistance(self.vi, ve)
        self.Si = np.array([y[0][np.argmin(y[1])], self.ta])
        
    def create_negihbor(self, ta):
        
        x = find_neighbors(self.vi, limit_x=[1, 548], limit_y=[1, 421])
        y = [[ta-2, ta-2]]*len(x)
        
        self.negihbor_Sij = dict(zip(x, y))
    
    def create_gi(self):
        self.gi_d = [self.td, self.ta]
        self.gi_v = [float('Inf')]
    
    def update_gi(self, i_x, i_y):
          
        i_x = np.array(i_x)
        i_y = np.array(i_y)
       
        f1 = StepFunction(self.gi_d, self.gi_v)
        
        f2 = StepFunction(i_x, i_y)
        
        if 0 not in i_x:
            f2.extend_domain(self.td)
        if 1080 not in i_x:
            f2.extend_domain(self.ta)
        
        x1 = np.sort(np.unique(np.append(self.gi_d, i_x)))
        y1 = [min(f1(i), f2(i)) if i in i_x else f1(i) for i in x1[:-1]]
        
        f = StepFunction(x1, y1)
        
        self.gi_d = f._x
        self.gi_v = f._y
    
        

In [11]:
def compute_minimum_cost(v_s, v_e, t_d, t_a, df):

    Q = {}
    QQ = []
    Q[v_s] = vertex_i(v_s, v_s, t_d, t_a)
    Q[v_s].gi_v = [0]
    Q[v_s].gi_d = [t_d, t_a]
    t = Q[v_s].Ti
    Q[v_s].update_TAUi_Si(v_e)
    Q[v_s].create_negihbor(t_a)
    
    
    Q[v_s].gi_d = [t_d, t_a]
    v_i = v_s
    run_times = 1
    while v_i != v_e:
        print Q[v_i].TAUi
        for i in find_neighbors(v_i, limit_x=[1, 548], limit_y=[1, 421]):
            if (Q[v_i].Si[0] < (t_a - 2)) and (Q[v_i].TAUi != float('Inf')):
                
                R_ij = [Q[v_i].Si[0], t_a-2]
                 
               
                x = [R_ij[0], Q[v_i].negihbor_Sij[i][0]]
                
                
                domain_ij = [x[0]+2, x[1]+2]
        
        
              
                if domain_ij[0] < domain_ij[1]:
                    f_ij = df[(df['xid'] == i[0]) & (df['yid'] == i[1])]
                    f_ij.sort_values(by='hour', ascending=True, inplace=True)
                
                #f_ij_d = np.append(f_ij.hour, t_a)
                #f_ij_v = np.where(f_ij.wind>=15, 1440, 0) 
                
                    f_ij = StepFunction(np.append(f_ij.hour, t_a), np.where(f_ij.wind>=15, 1440, 0))
                
                    f_ij = f_ij.find_interval_value(domain_ij)
                
                    update_d = f_ij[0]
                    update_v = f_ij[1] + Q[v_i].TAUi
                
                    Q[v_i].negihbor_Sij[i] = R_ij
                
                    if i not in [iii for iii in Q]:
                        Q[i] = vertex_i(i, v_s, t_d, t_a)
                        Q[i].create_gi()
                        Q[i].create_negihbor(t_a)
                    
                    #domain_j = [Q[i].Ti[0], Q[i].Si[0]]
                    
                
                    
                    if Q[i].Ti[0] < Q[i].Si[0]:
                        Q[i].update_gi(update_d, update_v)

                
                        run_times += 1
                        print run_times
                
                        Q[i].update_TAUi_Si(domain_ij)
                
 
                    if i not in [iii for iii, jjj in QQ]:
                        QQ.append((i, Q[i].TAUi))
                              
                
                
        if (Q[v_i].Ti[0] != Q[v_i].Si[0]) or (Q[v_i].Ti[1] != Q[v_i].Si[1]):
            
            domain_i = [Q[v_i].Ti[0], Q[v_i].Si[0]]
            
            Q[v_i].update_TAUi_Si(domain_i)
            
            
            QQ.append((v_i, Q[v_i].TAUi))
            
        
        QQ = sorted(QQ, key=operator.itemgetter(1), reverse=False)
        
        v_i = QQ.pop(0)[0]
    return Q

In [12]:

p_result = compute_minimum_cost((50, 50), (60, 60), 0, 1080, df)

0.0
2
3
4


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


5
0.0
6
7
8
0.0
9
10
11
0.0
12
13
14
0.0
15
16
17
0.0
0.0
18
19
20
0.0
21
22
0.0
23
24
0.0
25
26
27
0.0
28
29
0.0
30
31
32
0.0
33
34
0.0
35
36
37
0.0
38
39
40
0.0
41
42
0.0
43
44
0.0
45
46
0.0
47
48
0.0
49
50
51
0.0
52
53
0.0
54
55
0.0
56
57
58
0.0
59
60
0.0
61
62
0.0
63
64
65
0.0
66
67
68
0.0
69
70
0.0
71
72
0.0
73
74
0.0
75
76
0.0
77
78
0.0
79
80
0.0
81
82
83
0.0
84
85
0.0
86
87
0.0
88
89
0.0
90
91
92
0.0
93
94
0.0
95
96
0.0
97
98
0.0
99
100
101
0.0
102
103
104
0.0
105
106
0.0
107
108
0.0
109
110
0.0
111
112
0.0
113
114
0.0
115
116
0.0
117
118
0.0
119
120
0.0
121
122
123
0.0
124
125
0.0
126
127
0.0
128
129
0.0
130
131
0.0
132
133
134
0.0
135
136
0.0
137
138
0.0
139
140
0.0
141
142
0.0
143
144
145
0.0
146
147
148
0.0
149
150
0.0
151
152
0.0
153
154
0.0
155
156
0.0
157
158
0.0
159
160
0.0
161
162
0.0
163
164
0.0
165
166
0.0
167
168
0.0
169
170
171
0.0
172
173
0.0
174
175
0.0
176
177
0.0
178
179
0.0
180
181
0.0
182
183
184
0.0
185
186
0.0
187
188
0.0
189
190
0.0
191
192
0.0
193
194
0.0


1352
0.0
1353
1354
0.0
1355
1356
0.0
1357
1358
0.0
1359
1360
0.0
1361
1362
0.0
1363
1364
0.0
1365
1366
0.0
1367
1368
0.0
1369
1370
1371
0.0
1372
1373
0.0
1374
1375
0.0
1376
1377
0.0
1378
1379
0.0
1380
1381
0.0
1382
1383
0.0
1384
1385
0.0
1386
1387
0.0
1388
1389
0.0
1390
1391
0.0
1392
1393
0.0
1394
1395
0.0
1396
1397
0.0
1398
1399
0.0
1400
1401
0.0
1402
1403
0.0
1404
1405
0.0
1406
1407
1408
0.0
1409
1410
0.0
1411
1412
0.0
1413
1414
0.0
1415
1416
0.0
1417
1418
0.0
1419
1420
0.0
1421
1422
0.0
1423
1424
0.0
1425
1426
0.0
1427
1428
0.0
1429
1430
0.0
1431
1432
0.0
1433
1434
0.0
1435
1436
0.0
1437
1438
0.0
1439
1440
0.0
1441
1442
0.0
1443
1444
1445
0.0
1446
1447
1448
0.0
1449
1450
0.0
1451
1452
0.0
1453
1454
0.0
1455
1456
0.0
1457
1458
0.0
1459
1460
0.0
1461
1462
0.0
1463
1464
0.0
1465
1466
0.0
1467
1468
0.0
1469
1470
0.0
1471
1472
0.0
1473
1474
0.0
1475
1476
0.0
1477
1478
0.0
1479
1480
0.0
1481
1482
0.0
1483
1484
0.0
1485
1486
0.0
1487
1488
0.0
1489
1490
0.0
1491
1492
0.0
1493
1494
0.0
1495


In [13]:
print p_result[(60, 60)].Ti
print p_result[(60, 60)].Si

[  40 1080]
[  100.  1080.]


In [15]:
print p_result[(60, 60)].gi_d
print p_result[(60, 60)].gi_v

[    0.   100.   240.   360.   420.   480.  1080.]
[   inf     0.  1440.     0.  1440.     0.]


In [16]:
print p_result[(50, 51)].Ti
print p_result[(50, 51)].Si
print p_result[(50, 51)].gi_d
print p_result[(50, 51)].gi_v

[   2 1080]
[    2.  1080.]
[    0.    62.  1080.]
[ inf   0.]


In [17]:
print p_result[(50, 50)].Ti
print p_result[(50, 50)].Si
print p_result[(50, 50)].gi_d
print p_result[(50, 50)].gi_v

[   0 1080]
[    0.  1080.]
[0, 1080]
[0]


In [18]:
print p_result[(50, 51)].ta

1080


In [19]:
print p_result[(60, 60)].gi_d
print p_result[(60, 60)].gi_v

[    0.   100.   240.   360.   420.   480.  1080.]
[   inf     0.  1440.     0.  1440.     0.]


In [51]:
def path_selection(g, v_s, v_e, df):
    
    v_i = v_e
    path = []
    waiting_time = []

    while v_i != v_s:
        
        check_break = 0
        
        stop_find = 0
        
        for j in find_neighbors(v_i, limit_x=[1, 548], limit_y=[1, 421]):
            print '---------'
            print j
            check_break = 1
            
            
            gi = StepFunction(g[v_i].gi_d, g[v_i].gi_v)
            
            ti = g[v_i].gi_d[np.argmin(g[v_i].gi_v)]
            
            
            
            
            if (j in g) and (stop_find == 0):
               
                tj = g[j].gi_d[np.argmin(g[j].gi_v)]
                gj = StepFunction(g[j].gi_d, g[j].gi_v)
                f_ji = df[(df['xid'] == v_i[0]) & (df['yid'] == v_i[1])]
                f_ji.sort_values(by='hour', ascending=True, inplace=True)
                
                
                
                f_ji = StepFunction(np.append(f_ji.hour, g[j].ta), np.where(f_ji.wind>=15, 1440, 0))
     
                
                find_idx_j = (g[j].gi_v + f_ji(ti-2)) == gi(ti)
                
                find_idx_j = np.append(find_idx_j, [False])
              

                if sum(find_idx_j) > 0:
                    
                    find_idx_j = np.argmin(g[j].gi_d[find_idx_j])
                    tj = g[j].gi_d[find_idx_j]

                    if tj <= (ti-2):
                
                        path.append(j)
                        waiting_time.append(ti-2-tj)
                    
                        v_i = j
                        
                        check_break = 0
                        stop_find = 1
                        print check_break
                
            else:
                continue
        if check_break !=0:
            break
            
    path.reverse()
    waiting_time.reverse()
    return {'path': path, 'wating_time': waiting_time}    

In [52]:
path_selection(p_result, (50, 50), (55, 55), df)

---------
(54, 55)
0
---------
(55, 54)
---------
(55, 56)
---------
(56, 55)


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'path': [(54, 55)], 'wating_time': [78.0]}

In [125]:
print p_result[(60, 59)].gi_d
print p_result[(60, 59)].gi_v
print p_result[(60, 59)].Si
print p_result[(60, 59)].Ti

[    0.    98.  1080.]
[ inf   0.]
[   38.  1080.]
[  38 1080]


In [91]:
p_result[(60, 59)].update_TAUi_Si([38, 1080])

In [92]:
print p_result[(60, 59)].gi_d
print p_result[(60, 59)].gi_v
print p_result[(60, 59)].Si
print p_result[(60, 59)].Ti

[    0.    98.  1080.]
[ inf   0.]
[   98.  1080.]
[  38 1080]


In [115]:
sum(np.array([True, True, False]))

2